In [20]:
import fcsparser
import pandas as pd
import os
from glob import glob 
import numpy as np

In [23]:
fcs_folder = './fcs/'
fcs_pat = '.fcs'
csv_folder = './csv_data/Raw_csv'

if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)

def filterFiles(path, extension):
    '''
    Equivalent to:
    Files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(extension):
                Files.append(file)
    '''
    files = [file for root, dirs, files in os.walk(path) for file in files if file.endswith(extension)]
    for i in files:
        print(i)
    return files

In [24]:
files = filterFiles(fcs_folder, fcs_pat)

F04 17Sep-OD0_107-V10uLSlow.fcs
F09 17Sep-OD0_284-V10uLSlow.fcs
D11 17Sep-OD0_382-V10uLFast.fcs
H12.fcs
C04 17Sep-OD0_107-V5uLFast.fcs
C11 17Sep-OD0_382-V5uLFast.fcs
E03 17Sep-OD0_077-V5uLSlow.fcs
A06 OD=0.114.fcs
D03 17Sep-OD0_077-V10uLFast.fcs
E11 17Sep-OD0_382-V5uLSlow.fcs
E10 17Sep-OD0_320-V5uLSlow.fcs
C05 17Sep-OD0_143-V5uLFast.fcs
F03 17Sep-OD0_077-V10uLSlow.fcs
C06 17Sep-OD0_171-V5uLFast.fcs
F10 17Sep-OD0_320-V10uLSlow.fcs
C03 17Sep-OD0_077-V5uLFast.fcs
A10 OD=0.218.fcs
A01 OD=0.028.fcs
D04 17Sep-OD0_107-V10uLFast.fcs
F11 17Sep-OD0_382-V10uLSlow.fcs
E09 17Sep-OD0_284-V5uLSlow.fcs
H11 H20After.fcs
D06 17Sep-OD0_171-V10uLFast.fcs
A11 OD=0.261.fcs
D08 17Sep-OD0_235-V10uLFast.fcs
E07 17Sep-OD0_198-V5uLSlow.fcs
D01 17Sep-OD0_024-V10uL.fcs
F02 17Sep-OD0_045-V10uLSlow.fcs
A08 OD=0.153.fcs
D05 17Sep-OD0_143-V10uLFast.fcs
F06 17Sep-OD0_171-V10uLSlow.fcs
F08 17Sep-OD0_235-V10uLSlow.fcs
C08 17Sep-OD0_235-V5uLFast.fcs
F07 17Sep-OD0_198-V10uLSlow.fcs
C09 17Sep-OD0_284-V5uLFast.fcs
C02 17Sep-

In [25]:
for i in files:
    path = fcs_folder + i
    meta, data = fcsparser.parse(path, meta_data_only=False, reformat_meta=True)
    data.columns=['FSC-A','SSC-A','FL1-A','FL2-A','FL3-A','FL4-A','FSC-H','SSC-H','FL1-H','FL2-H','FL3-H','FL4-H','Width','Time']
    data.to_csv(csv_folder +'/'+ i[:-3]+'csv',index=False)

In [65]:
files_csv = filterFiles(csv_folder, '.csv')

D06 17Sep-OD0_171-V10uLFast.csv
C01 17Sep-OD0_024-V5uL.csv
C05 17Sep-OD0_143-V5uLFast.csv
C11 17Sep-OD0_382-V5uLFast.csv
E03 17Sep-OD0_077-V5uLSlow.csv
E07 17Sep-OD0_198-V5uLSlow.csv
F07 17Sep-OD0_198-V10uLSlow.csv
E02 17Sep-OD0_045-V5uLSlow.csv
F05 17Sep-OD0_143-V10uLSlow.csv
F04 17Sep-OD0_107-V10uLSlow.csv
D11 17Sep-OD0_382-V10uLFast.csv
A05 OD=0.088.csv
D01 17Sep-OD0_024-V10uL.csv
C03 17Sep-OD0_077-V5uLFast.csv
A10 OD=0.218.csv
E11 17Sep-OD0_382-V5uLSlow.csv
E05 17Sep-OD0_143-V5uLSlow.csv
C04 17Sep-OD0_107-V5uLFast.csv
F11 17Sep-OD0_382-V10uLSlow.csv
E09 17Sep-OD0_284-V5uLSlow.csv
A08 OD=0.153.csv
H09 agua.csv
D04 17Sep-OD0_107-V10uLFast.csv
E04 17Sep-OD0_107-V5uLSlow.csv
A01 OD=0.028.csv
F10 17Sep-OD0_320-V10uLSlow.csv
C02 17Sep-OD0_045-V5uLFast.csv
H10.csv
A03 OD=0.054OJO.csv
F02 17Sep-OD0_045-V10uLSlow.csv
A02 OD=0.055.csv
D02 17Sep-OD0_045-V10uLFast.csv
D09 17Sep-OD0_284-V10uLFast.csv
F08 17Sep-OD0_235-V10uLSlow.csv
H11 H20After.csv
E08 17Sep-OD0_235-V5uLSlow.csv
F06 17Sep-OD0_1

In [66]:
#defines dir
csv_save = './csv_data/Corrected_csv/'
#creates dir
if not os.path.exists(csv_save):
    os.makedirs(csv_save)
    
#create stats df
columns = ['label', 'mFSC-H', 'stdFSC-H', 'mFL3-A', 'stdFL3-A']
index = [i for i in range(len(files))]
stats_df = pd.DataFrame(columns=columns, index = index)
#for each csv file
for j in range(len(files_csv)):
    #creates a dataframe
    df = pd.read_csv(csv_folder+'/'+ files_csv[j])
    ind = []
    #for each line of the dataFrame
    for i in range(len(df)):
        #Apply the filter and stores the indices
        if df['SSC-A'][i] == 0 or df['FL3-A'][i] < 800 or df['FSC-H'][i] < 11000:
            ind.append(i)
    #drop the lines corresponding to the indices and reset indices
    df = df.drop(df.index[ind])
    df = df.reset_index(drop=True)
    #the name of the new DataFrame is the first three letters (cell label)
    df['name'] = files_csv[j][0:3]
    #Make statistics and creates a column with the results (this is redundant)
    sigmasqH = np.std(df['FSC-H'].tolist())
    promH = np.mean(df['FSC-H'].tolist())
    fanofact =sigmasqH/promH
    df['cvFSC-H'] = fanofact
    df['mFSC-H'] = promH
    sigmasq3 = np.std(df['FL3-A'].tolist())
    prom3 = np.mean(df['FL3-A'].tolist())
    fanofact =sigmasq3/prom3
    df['cvFL3-A'] = fanofact
    df['mFL3-A'] = prom3
    #Save the csv with the filter and statistics
    df.to_csv(csv_save  +files_csv[j][0:3]+'.csv',index=False)
    print('File ' + str(j+1) + ' out of ' + str(len(files_csv)) + ' Done')    
    stats_df.loc[j] = [files_csv[j][0:3], promH, sigmasqH, prom3, sigmasq3]

File 1 out of 57 Done
File 2 out of 57 Done
File 3 out of 57 Done
File 4 out of 57 Done
File 5 out of 57 Done
File 6 out of 57 Done
File 7 out of 57 Done
File 8 out of 57 Done
File 9 out of 57 Done
File 10 out of 57 Done
File 11 out of 57 Done
File 12 out of 57 Done
File 13 out of 57 Done
File 14 out of 57 Done
File 15 out of 57 Done
File 16 out of 57 Done
File 17 out of 57 Done
File 18 out of 57 Done
File 19 out of 57 Done
File 20 out of 57 Done
File 21 out of 57 Done
File 22 out of 57 Done
File 23 out of 57 Done
File 24 out of 57 Done
File 25 out of 57 Done
File 26 out of 57 Done
File 27 out of 57 Done
File 28 out of 57 Done
File 29 out of 57 Done
File 30 out of 57 Done
File 31 out of 57 Done
File 32 out of 57 Done
File 33 out of 57 Done
File 34 out of 57 Done
File 35 out of 57 Done
File 36 out of 57 Done
File 37 out of 57 Done
File 38 out of 57 Done
File 39 out of 57 Done
File 40 out of 57 Done
File 41 out of 57 Done
File 42 out of 57 Done
File 43 out of 57 Done
File 44 out of 57 Do

In [94]:
#Statistics saved in statistics.csv
stats_df = stats_df.sort_values('label')
stats_df = stats_df.reset_index(drop=True)
csv_stats_dir = './csv_data/Statistics_csv/'
if not os.path.exists(csv_stats_dir):
    os.makedirs(csv_stats_dir)
stats_df.to_csv(csv_stats_dir + 'statistics'+'.csv', index = False)

In [19]:
csv_folder = './csv_data/Corrected_csv/'
files = filterFiles(csv_folder, '.csv')

df = pd.read_csv(csv_folder + files[0])
for i in files[1:]:
    df1 = pd.read_csv(csv_folder + i)
    df = df.append(df1)
    
csv_folder_final = './csv_data/Merged/'
if not os.path.exists(csv_folder_final):
    os.makedirs(csv_folder_final)
df.to_csv(csv_folder_final +'mergedData'+'.csv',index=False)